In [1]:
import numpy as np
import helper_functions as hf
from scipy.stats import kurtosis, skew
from pywt import dwt
from biosppy.signals.eeg import eeg
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC


In [2]:
X_train_eeg1, _ = hf.read_csv_to_matrix("task5/input/train_eeg1.csv", "Id")
X_train_eeg2, _ = hf.read_csv_to_matrix("task5/input/train_eeg2.csv", "Id")
X_train_emg, _ = hf.read_csv_to_matrix("task5/input/train_emg.csv", "Id")
X_test_eeg1, _ = hf.read_csv_to_matrix("task5/input/test_eeg1.csv", "Id")
X_test_eeg2, _ = hf.read_csv_to_matrix("task5/input/test_eeg2.csv", "Id")
X_test_emg, _ = hf.read_csv_to_matrix("task5/input/test_emg.csv", "Id")
y_train, _ = hf.read_csv_to_matrix("task5/input/train_labels.csv", "Id")
_, test_index = hf.read_csv_to_matrix("task5/input/sample.csv", "Id")
y_train = np.squeeze(y_train)


In [10]:
def feature_extraction(signal):
    # ensure numpy
    signal = np.array(signal)

    # mean
    mean = np.mean(signal)

    # median
    median = np.median(signal)

    # maximum amplitude
    maxAmp = np.abs(signal - mean).max()

    # variance
    sigma2 = signal.var(ddof=1)

    # standard deviation
    sigma = signal.std(ddof=1)

    # absolute deviation
    ad = np.sum(np.abs(signal - median))

    # kurtosis
    kurt = kurtosis(signal, bias=False)

    # skweness
    skewness = skew(signal, bias=False)

    return np.r_[mean, median, maxAmp, sigma2, sigma, ad, kurt, skewness]


def eeg_feature_extraction(x):
    x_new = []

    for idx in range(x.shape[0]):
        analysis = eeg(signal=x[idx].reshape(-1, x.shape[1]).transpose(), sampling_rate=128, show=False)
        v1 = analysis['filtered'].transpose().reshape(-1)
        v2 = analysis['theta'].transpose().reshape(-1)
        v3 = analysis['alpha_low'].transpose().reshape(-1)
        v4 = analysis['alpha_high'].transpose().reshape(-1)
        v5 = analysis['beta'].transpose().reshape(-1)
        v6 = analysis['gamma'].transpose().reshape(-1)
        x_new.append(np.r_[v1, v2, v3, v4, v5, v6, feature_extraction(v1), feature_extraction(v2), feature_extraction(v3), feature_extraction(v4), feature_extraction(v5), feature_extraction(v6)])
    return np.asarray(x_new)


def emg_feature_extraction(x):
    x_new = []
    for idx in range(x.shape[0]):
        cA, cD = dwt(data=x[idx], wavelet='db2')
        to_append = np.r_[feature_extraction(x), cA, cD, feature_extraction(cA), feature_extraction(cD)]
        x_new.append(to_append)

    return np.asarray(x_new)


In [8]:
print("==> Train feature extraction")
print("==> EEG1")
X_train_eeg1_new = eeg_feature_extraction(X_train_eeg1)


==> Train feature extraction
==> EEG1


In [9]:
print("==> EEG2")
X_train_eeg2_new = eeg_feature_extraction(X_train_eeg2)


==> EEG2


In [11]:
print("==> EMG")
X_train_emg_new = emg_feature_extraction(X_train_emg)


==> EMG


In [ ]:
# Fuse extracted features
X_train = np.c_[X_train_eeg1_new, X_train_eeg2_new, X_train_emg_new]


==> Train feature extraction
==> EEG1


C:\Users\Frankie\PycharmProjects\global_venv_36\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


ValueError: setting an array element with a sequence.

In [ ]:
print("==> PCA")
pca = PCA(
    n_components=10,
    whiten=True
)

X_train = pca.fit_transform(X_train)


==> Train feature extraction
==> EEG1


C:\Users\Frankie\PycharmProjects\global_venv_36\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


ValueError: setting an array element with a sequence.

In [ ]:
print("==> Initialize classifier")
clf = SVC(
    C=1.0,
    kernel='rbf',
    shrinking=True,
    probability=False,
    class_weight='balanced'
)


==> Train feature extraction
==> EEG1


C:\Users\Frankie\PycharmProjects\global_venv_36\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


ValueError: setting an array element with a sequence.

In [ ]:
print("==> 3 fold crossvalidation")
scores = cross_val_score(
    estimator=clf,
    X=X_train,
    y=y_train,
    groups=None,
    scoring=hf.scorer(),
    cv=3,
    n_jobs=3,
    verbose=0,
    fit_params=None,
    pre_dispatch='2*n_jobs',
)

print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))


==> Train feature extraction
==> EEG1


C:\Users\Frankie\PycharmProjects\global_venv_36\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


ValueError: setting an array element with a sequence.